In [1]:
import pandas as pd
from textblob import TextBlob
import string


In [2]:
df = pd.read_csv("Data/train.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
id       31962 non-null int64
label    31962 non-null int64
tweet    31962 non-null object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [4]:
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
import re 

def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)


def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet


def process_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', '', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            processed_tweet.append(word)

    return ' '.join(processed_tweet)

In [6]:
df['processed_tweets'] = df['tweet'].apply(process_tweet)
df.drop('tweet',axis=1,inplace=True)

In [7]:
df['processed_tweets'].head()

0    when a father is dysfunctional and is so selfi...
1    thanks for lyft credit i cant use cause they d...
2                                  bihday your majesty
3           model i love u take with u all the time in
4                    factsguide society now motivation
Name: processed_tweets, dtype: object

In [8]:
df['char_count'] = df['processed_tweets'].apply(len)
df['word_count'] = df['processed_tweets'].apply(lambda x: len(x.split()))
df['word_density'] = df['char_count'] / (df['word_count']+1)

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
df.head()

,id,label,processed_tweets,char_count,word_count,word_density
0,1,0,when a father is dysfunctional and is so selfi...,91,17,5.055556
1,2,0,thanks for lyft credit i cant use cause they d...,101,17,5.611111
2,3,0,bihday your majesty,19,3,4.750000
3,4,0,model i love u take with u all the time in,42,11,3.500000
4,5,0,factsguide society now motivation,33,4,6.600000


In [11]:
TextBlob(df['processed_tweets'][0]).tags

[('when', 'WRB'),
 ('a', 'DT'),
 ('father', 'NN'),
 ('is', 'VBZ'),
 ('dysfunctional', 'JJ'),
 ('and', 'CC'),
 ('is', 'VBZ'),
 ('so', 'RB'),
 ('selfish', 'JJ'),
 ('he', 'PRP'),
 ('drags', 'VBZ'),
 ('his', 'PRP$'),
 ('kids', 'NNS'),
 ('into', 'IN'),
 ('his', 'PRP$'),
 ('dysfunction', 'NN'),
 ('run', 'VB')]

In [12]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    wiki = TextBlob(x)
    for tup in wiki.tags:
        ppo = list(tup)[1]
        if ppo in pos_family[flag]:
            cnt += 1

    return cnt

df['noun_count'] = df['processed_tweets'].apply(lambda x: check_pos_tag(x, 'noun'))
df['verb_count'] = df['processed_tweets'].apply(lambda x: check_pos_tag(x, 'verb'))
df['adj_count'] = df['processed_tweets'].apply(lambda x: check_pos_tag(x, 'adj'))
df['adv_count'] = df['processed_tweets'].apply(lambda x: check_pos_tag(x, 'adv'))
df['pron_count'] = df['processed_tweets'].apply(lambda x: check_pos_tag(x, 'pron'))

In [13]:
df.head()

,id,label,processed_tweets,char_count,word_count,word_density,noun_count,verb_count,adj_count,adv_count,pron_count
0,1,0,when a father is dysfunctional and is so selfi...,91,17,5.055556,3,4,2,2,3
1,2,0,thanks for lyft credit i cant use cause they d...,101,17,5.611111,7,5,2,0,1
2,3,0,bihday your majesty,19,3,4.750000,1,0,0,1,1
3,4,0,model i love u take with u all the time in,42,11,3.500000,4,1,2,0,0
4,5,0,factsguide society now motivation,33,4,6.600000,2,0,1,1,0


In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['char_count','word_count','word_density', 'noun_count','verb_count','adj_count','adv_count','pron_count']] = \
 scaler.fit_transform(df[['char_count','word_count','word_density', 'noun_count','verb_count','adj_count','adv_count','pron_count']])

In [15]:
df.head()

,id,label,processed_tweets,char_count,word_count,word_density,noun_count,verb_count,adj_count,adv_count,pron_count
0,1,0,when a father is dysfunctional and is so selfi...,0.659420,0.53125,0.324074,0.166667,0.363636,0.166667,0.222222,0.375
1,2,0,thanks for lyft credit i cant use cause they d...,0.731884,0.53125,0.359687,0.388889,0.454545,0.166667,0.000000,0.125
2,3,0,bihday your majesty,0.137681,0.09375,0.304487,0.055556,0.000000,0.000000,0.111111,0.125
3,4,0,model i love u take with u all the time in,0.304348,0.34375,0.224359,0.222222,0.090909,0.166667,0.000000,0.000
4,5,0,factsguide society now motivation,0.239130,0.12500,0.423077,0.111111,0.000000,0.083333,0.111111,0.000


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
y = df['label']
df.drop('id',axis=1,inplace=True)
df.drop('label',axis=1,inplace=True)
X = df
#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [17]:
X.head()

,processed_tweets,char_count,word_count,word_density,noun_count,verb_count,adj_count,adv_count,pron_count
0,when a father is dysfunctional and is so selfi...,0.659420,0.53125,0.324074,0.166667,0.363636,0.166667,0.222222,0.375
1,thanks for lyft credit i cant use cause they d...,0.731884,0.53125,0.359687,0.388889,0.454545,0.166667,0.000000,0.125
2,bihday your majesty,0.137681,0.09375,0.304487,0.055556,0.000000,0.000000,0.111111,0.125
3,model i love u take with u all the time in,0.304348,0.34375,0.224359,0.222222,0.090909,0.166667,0.000000,0.000
4,factsguide society now motivation,0.239130,0.12500,0.423077,0.111111,0.000000,0.083333,0.111111,0.000


In [42]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest, chi2, f_classif

count_vect = CountVectorizer(df['processed_tweets'], stop_words='english', ngram_range=(1,3), max_df=0.9, strip_accents="unicode")
tfidf = TfidfVectorizer(use_idf=True)
tfidf = tfidf.fit(df['processed_tweets'])
tfidf = tfidf.transform(X)
svc = LinearSVC()
type(tfidf)
svc.fit(X, tfidf)


# text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
#                       ('tfidf', TfidfTransformer()),
#                      ('chi2', SelectKBest(chi2)),
#                      ('clf', LinearSVC(verbose=True, )),])

scipy.sparse.csr.csr_matrix

In [19]:
params = {"vect__ngram_range": [(1, 3)],
          "vect__max_df": [0.9],
          "vect__strip_accents": ["unicode"],
        "tfidf__use_idf": [True],
          "chi2__k" : [25000],
          "clf__C": [2.4],
          "clf__max_iter" : [6],
          "clf__loss" : ["hinge"],
          "clf__tol":[0.00001]
         }

# #Random Forest

# params = {"vect__ngram_range": [(1, 2)],
#           "vect__max_df": [0.1],
#           "vect__strip_accents": ["ascii"],
#         "tfidf__use_idf": [True],
#           "chi2__k" :[27000],
#          }

In [20]:
from sklearn.model_selection import GridSearchCV
text_clf = GridSearchCV(text_clf, params, verbose=2, n_jobs=-1)

In [30]:
y = y.to_frame()


AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [36]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 9 columns):
processed_tweets    31962 non-null object
char_count          31962 non-null float64
word_count          31962 non-null float64
word_density        31962 non-null float64
noun_count          31962 non-null float64
verb_count          31962 non-null float64
adj_count           31962 non-null float64
adv_count           31962 non-null float64
pron_count          31962 non-null float64
dtypes: float64(8), object(1)
memory usage: 2.2+ MB


In [35]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 1 columns):
label    31962 non-null int64
dtypes: int64(1)
memory usage: 249.8 KB


In [28]:
model = text_clf.fit(X,y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\ProgramData\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\ProgramData\Anaconda3\lib\runpy.py

In [ ]:
model.best_params_

In [ ]:
predicted = model.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_test, predicted)

In [ ]:
submission = pd.read_csv('Data/test.csv')
submission.info()

submission['processed_tweet'] = submission['tweet'].apply(process_tweet)
submission.head()

submission.drop('tweet', axis=1,inplace=True)
submission.head()

submission['char_count'] = submission['processed_tweets'].apply(len)
submission['word_count'] = submission['processed_tweets'].apply(lambda x: len(x.split()))
submission['word_density'] = submission['char_count'] / (submission['word_count']+1)
submission['noun_count'] = submission['processed_tweets'].apply(lambda x: check_pos_tag(x, 'noun'))
submission['verb_count'] = submission['processed_tweets'].apply(lambda x: check_pos_tag(x, 'verb'))
submission['adj_count'] = submission['processed_tweets'].apply(lambda x: check_pos_tag(x, 'adj'))
submission['adv_count'] = submission['processed_tweets'].apply(lambda x: check_pos_tag(x, 'adv'))
submission['pron_count'] = submission['processed_tweets'].apply(lambda x: check_pos_tag(x, 'pron'))

predicted = text_clf.predict(submission)

predicted

final_predict = pd.DataFrame(predicted,columns=['label'])
result = pd.DataFrame(submission['id'],columns=['id'])
result = pd.concat([result,final_predict],axis=1)
result.to_csv('final_predictions.csv',index=False)

result['label'].value_counts()